# A Network Tour of Data Science
### &nbsp; &nbsp; &nbsp; Xavier Bresson, Winter 2016/17
## Exercise 4 : Introduction to TensorFlow

In [1]:
# Import libraries
import tensorflow as tf
import numpy as np
import time
import collections
import os

In [2]:
# Import MNIST data with TensorFlow
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(os.path.join('datasets', 'mnist'), one_hot=True) # load data in local folder

train_data = mnist.train.images.astype(np.float32)
train_labels = mnist.train.labels

test_data = mnist.test.images.astype(np.float32)
test_labels = mnist.test.labels

print(train_data.shape)
print(train_labels.shape)
print(test_data.shape)
print(test_labels.shape)

Extracting datasets/mnist/train-images-idx3-ubyte.gz
Extracting datasets/mnist/train-labels-idx1-ubyte.gz
Extracting datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting datasets/mnist/t10k-labels-idx1-ubyte.gz
(55000, 784)
(55000, 10)
(10000, 784)
(10000, 10)


## 1st Step: Construct Computational Graph

### Question 1: Prepare the input variables (x,y_label) of the computational graph

Hint: You may use the function *tf.placeholder()*

In [3]:
# computational graph inputs
batch_size = 100
d = train_data.shape[1]
nc = 10
x = tf.placeholder(tf.float32,[batch_size,d]); print('x=',x,x.get_shape())
y_label = tf.placeholder(tf.float32,[batch_size,nc]); print('y_label=',y_label,y_label.get_shape())

x= Tensor("Placeholder:0", shape=(100, 784), dtype=float32) (100, 784)
y_label= Tensor("Placeholder_1:0", shape=(100, 10), dtype=float32) (100, 10)


### Question 2: Prepare the variables (W,b) of the computational graph

Hint: You may use the function *tf.Variable(), tf.truncated_normal()*

In [4]:
# computational graph variables
initial = tf.truncated_normal([d,nc], stddev=0.1); W = tf.Variable(initial); print('W=',W.get_shape())
b = tf.Variable(tf.zeros([nc],tf.float32)); print('b=',b.get_shape())

W= (784, 10)
b= (10,)


### Question 3: Compute the classifier such that
$$
y=softmax(Wx +b)
$$

Hint: You may use the function *tf.matmul(), tf.nn.softmax()*

In [5]:
# Construct CG / output value
y = tf.matmul(x, W); print('y1=',y,y.get_shape())
y += b; print('y2=',y,y.get_shape())
y = tf.nn.softmax(y); print('y3=',y,y.get_shape())

y1= Tensor("MatMul:0", shape=(100, 10), dtype=float32) (100, 10)
y2= Tensor("add:0", shape=(100, 10), dtype=float32) (100, 10)
y3= Tensor("Softmax:0", shape=(100, 10), dtype=float32) (100, 10)


### Question 4: Construct the loss of the computational graph such that
$$
loss = cross\ entropy(y_{label},y) =  mean_{all\ data} \ \sum_{all\ classes} -\ y_{label}.\log(y)
$$

Hint: You may use the function *tf.Variable(), tf.truncated_normal()*

In [6]:
# Loss
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(y), 1))

### Question 5: Construct the L2 regularization of (W,b) to the computational graph such that
$$
R(W) = \|W\|_2^2\\
R(b) = \|b\|_2^2
$$

Hint: You may use the function *tf.nn.l2_loss()*

In [7]:
reg_loss = tf.nn.l2_loss(W)
reg_loss += tf.nn.l2_loss(b)

### Question 6: Form the total loss
$$
total\ loss = cross\ entropy(y_{label},y) + reg\_par* (R(W) + R(b))
$$

In [8]:
reg_par = 1e-3
total_loss = cross_entropy + reg_par* reg_loss

### Question 7: Perform optimization of the total loss for learning weight variables of the computational graph

Hint: You may use the function *tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)*

In [9]:
# Update CG variables / backward pass
train_step = tf.train.GradientDescentOptimizer(0.25).minimize(total_loss)

### Question 8: Evaluate the accuracy

Hint: You may use the function *tf.equal(tf.argmax(y,1), tf.argmax(y_label,1))* and *tf.reduce_mean()*

In [10]:
# Accuracy
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_label,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## 2nd Step: Run the Computational Graph with batches of training data
Check out the accuracy of test set

In [11]:
# Create test set 
idx = np.random.permutation(test_data.shape[0]) # rand permutation
idx = idx[:batch_size]
test_x, test_y = test_data[idx,:], test_labels[idx]

In [12]:
n = train_data.shape[0]
indices = collections.deque()

# Running Computational Graph
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)
for i in range(50):
    
    # Batch extraction
    if len(indices) < batch_size:
        indices.extend(np.random.permutation(n)) # rand permutation
    idx = [indices.popleft() for i in range(batch_size)] # extract n_batch data
    batch_x, batch_y = train_data[idx,:], train_labels[idx]
    
    # Run CG for variable training
    _,acc_train,total_loss_o = sess.run([train_step,accuracy,total_loss], feed_dict={x: batch_x, y_label: batch_y})
    print('\nIteration i=',i,', train accuracy=',acc_train,', loss=',total_loss_o)
    
    # Run CG for testset
    acc_test = sess.run(accuracy, feed_dict={x: test_x, y_label: test_y})
    print('test accuracy=',acc_test)


Iteration i= 0 , train accuracy= 0.1 , loss= 2.61576
test accuracy= 0.16

Iteration i= 1 , train accuracy= 0.12 , loss= 2.25159
test accuracy= 0.33

Iteration i= 2 , train accuracy= 0.3 , loss= 2.16757
test accuracy= 0.36

Iteration i= 3 , train accuracy= 0.38 , loss= 1.88768
test accuracy= 0.51

Iteration i= 4 , train accuracy= 0.5 , loss= 1.70732
test accuracy= 0.55

Iteration i= 5 , train accuracy= 0.59 , loss= 1.52506
test accuracy= 0.62

Iteration i= 6 , train accuracy= 0.53 , loss= 1.57437
test accuracy= 0.68

Iteration i= 7 , train accuracy= 0.62 , loss= 1.34377
test accuracy= 0.69

Iteration i= 8 , train accuracy= 0.64 , loss= 1.36525
test accuracy= 0.72

Iteration i= 9 , train accuracy= 0.68 , loss= 1.25514
test accuracy= 0.72

Iteration i= 10 , train accuracy= 0.63 , loss= 1.22931
test accuracy= 0.68

Iteration i= 11 , train accuracy= 0.66 , loss= 1.17098
test accuracy= 0.74

Iteration i= 12 , train accuracy= 0.68 , loss= 1.24049
test accuracy= 0.75

Iteration i= 13 , train 